In [2]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 색온도 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('./../cct_swr_test.csv', delimiter=',', dtype='float')

x_data = import_data[:, :25]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
test_x = x_data[85:, :25]
x_data = x_data[:85, :25]

y_data = []
test_y = []

for i in range(85):
    temp = []
    temp.append(import_data[i][25])
    y_data.append(temp)

raw_y = y_data
    
for i in range(85, len(import_data)):
    temp = []
    temp.append(import_data[i][25])
    test_y.append(temp)
    
# raw_y = y_data

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 25])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

#기존 코드
# W = tf.Variable(tf.random_normal([25, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[25, nb_classes], initializer=xavier_init(25, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(3701):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
        print(a[0][0], ', ', a[0][1], ', ', a[0][2], ', ', a[0][3], ', ', a[0][4])
#         if(sess.run(tf.argmax(a, 1)) ==raw_y[i]):
#             accuracy += 1
#     print("accuracy = ", float(accuracy / len(x_data) * 100))

0 1.4309957
100 1.00689
200 0.86928684
300 0.7885412
400 0.7311735
500 0.6860948
600 0.64866585
700 0.61658055
800 0.5885212
900 0.56364465
1000 0.54136044
1100 0.52122563
1200 0.50289357
1300 0.48608592
1400 0.4705768
1500 0.4561809
1600 0.44274643
1700 0.43014777
1800 0.41828096
1900 0.40705943
2000 0.3964105
2100 0.38627288
2200 0.37659472
2300 0.36733156
2400 0.35844526
2500 0.34990287
2600 0.34167576
2700 0.3337387
2800 0.32606983
2900 0.31864944
3000 0.3114603
3100 0.30448705
3200 0.2977159
3300 0.29113442
3400 0.2847315
3500 0.278497
3600 0.27242175
3700 0.2664976
-----------------------------
train_data =  85 test_data =  1
[4] [4.0]
3.8918993e-13 ,  9.20248e-11 ,  1.4528635e-05 ,  0.15587011 ,  0.84411544
